In [1]:
import wikipedia
import re
from pymystem3 import Mystem
from tqdm import tqdm
from multiprocessing import Pool
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

wikipedia.set_lang('ru')

## Get the data

In [2]:
page_names = ['Трезубец_Посейдона', 'Флаг Барбадоса', 'Биологическое_оружие',
'Блэкберн,_Люк_Прайор', 'Рим', 'Аппиева_дорога_при_закате_солнца']

wikipedia_pages = [wikipedia.page(page_name) for page_name in page_names]

In [3]:
queries = [
    'Пейзаж с выжженной равниной увековечил образ вечного города.',
    'Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.',
    'Трезубец на жёлто-синем фоне — это флаг Барбадоса.'
]

## Stemming

In [7]:
docs = []
original_sentences = []
m = Mystem()
for page in wikipedia_pages:
    s = re.sub('\s', ' ', page.content) # remove newlines and tabs
    s = re.sub('\=\= Примечания[\w\s\=]*', '', s) # remove all text after 'Примечания'
    s = re.sub(r'\=\=.*\=\=', ' ', s) # remove headers
    s = re.sub(r'[^\w \.]', '', s.lower()) # remove non-letter symbols
    s = [doc.strip() for doc in s.split('.')]
    if
    original_sentences += s
    with Pool(4) as p:
        lemmitized = list(tqdm(p.imap(m.lemmatize, s), total=len(s)))
    # lemmitized = [m.lemmatize(doc) for doc in s]
    docs += [[lemma for lemma in doc if lemma.isalpha() or lemma.isdigit()] for doc in lemmitized]

Installing mystem to C:\Users\admin/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip
100%|██████████████████████████████████████████| 31/31 [00:28<00:00,  1.10it/s]


In [8]:
queries_processed = []
for query in tqdm(queries):
    s = m.lemmatize(query.lower())
    s = [lemma for lemma in s if lemma.isalpha() or lemma.isdigit()]
    queries_processed.append(s)

100%|████████████████████████████████████████████| 3/3 [00:05<00:00,  1.98s/it]


In [10]:
SAVE_DATA = False
if SAVE_DATA:
    with open('lemmatized.pkl', 'wb') as output_file:
        pickle.dump((docs, original_sentences, queries_processed), output_file)

In [14]:
with open('lemmatized.pkl', 'rb') as output_file:
    docs, original_sentences, queries_processed = pickle.load(output_file)

## TF-iDF

In [15]:
def vectorize(docs, vocab=None):
    '''
    docs - list of lists of tokens
    vocab - set of tokens
    Returns count array with shape (N x V), where N - number of documents and V - size of vocab
    '''
    if vocab is None:
        vocab = set([token for sentence in docs for token in sentence if len(token)>1])
    vectorized = np.zeros((len(docs),len(vocab)))

    token_ind_dict = {tok: ind for ind, tok in dict(enumerate(vocab)).items()}

    for doc_ind, doc in enumerate(docs):
        for token in doc:
            try:
                token_id = token_ind_dict[token]
                vectorized[doc_ind][token_id] += 1
            except KeyError:
                pass
    return vectorized

def get_tf(matrix):
    return matrix

def get_tf_norm(matrix):
    return np.nan_to_num(matrix / matrix.sum(axis=0))

def get_tf_max_norm(matrix, alpha=0.4):
    tf =  alpha + (1-alpha) * np.nan_to_num(matrix / matrix.max(axis=1)[:,None])
    tf[tf == alpha] = 0
    return tf

def get_idf(matrix, smoothing=3):
    return matrix.shape[0] / ((matrix > 0).sum(axis=0) + 1)
    # return np.nan_to_num(matrix.shape[0] / ((matrix > 0).sum(axis=0)))

Remove short tokens and sentences without tokens

In [8]:
# docs = [tokens for tokens, sent in zip(docs, original_sentences) if len(sent)>5]
# original_sentences = [sent for sent in original_sentences if len(sent)>5]

# original_sentences = [sent for tokens, sent in zip(docs, original_sentences) if len(tokens)>0]
# docs = [tokens for tokens in docs if len(tokens)>0]

### No TF normalization

In [16]:
vocab = set([token for sentence in docs for token in sentence if len(token)>1])
doc_vec = vectorize(docs, vocab)
doc_tf = get_tf(doc_vec)
doc_idf = get_idf(doc_vec)

In [17]:
queries_vec = vectorize(queries_processed, vocab)
queries_tf = get_tf(queries_vec)
# queries_idf = get_idf(queries_vec)

In [18]:
X = doc_tf * doc_idf
X = normalize(X, axis=1)
queries_tfidf = queries_tf * doc_idf
query_results = cosine_similarity(X, queries_tfidf)
query_results.argmax(axis=0)

array([101,  55,  44], dtype=int64)

In [19]:
for i, query in enumerate(queries):
    print(f"Query:\n{query}\n")
    print("Results:\n  Weight  | Sentence")
    for ind in query_results[:,i].argsort()[:-4:-1]:
        print(f"({query_results[ind,i]:.6f})  {original_sentences[ind]}")
        # print(f"With weight {query_results[ind,i]:.6f}")
    print("------\n")

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight  | Sentence
(0.550276)  рим стали часто называть вечным лат
(0.079148)  также рим называют городом на семи холмах
(0.055470)  рим  один из старейших городов мира древняя столица римской империи
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight  | Sentence
(0.309441)  биологическое оружие
(0.256880)  его обвинили в заговоре с целью убийства и в нарушении нейтралитета канады но освободили под залог в 8000 долларов
(0.144803)  в 1879 году он был избран губернатором кентукки от демократической партии
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight  | Sentence
(0.594446)  флаг барбадоса  официальный символ государства барбадос
(0.101780)  посейдон со своей женой богиней амфитритой и сыном тритоном обитают в роскошном дворце на дне моря в окружении нереид гиппокампов и других обитателей моря мчится по

Резульаты похожи на то, что выдает реализация TF-iDF из пакета sklearn

### Max TF normalization

In [20]:
alpha = 0.4

doc_tf_2 = get_tf_max_norm(doc_vec, alpha=alpha)
queries_tf_2 = get_tf_max_norm(queries_vec, alpha=alpha)

C:\Python36\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


In [21]:
X_2 = doc_tf_2 * doc_idf
# X_2 =  normalize(X_2)
queries_tfidf_2 = queries_tf_2 * doc_idf
# queries_tfidf_2 = normalize(queries_tfidf_2, axis=1)
query_results_2 = cosine_similarity(X_2, queries_tfidf_2)
query_results_2.argmax(axis=0)

array([101,  55,  44], dtype=int64)

In [22]:
for i, query in enumerate(queries):
    print(f"Query:\n{query}\n")
    print("Results:\n  Weight  | Sentence")
    for ind in query_results_2[:,i].argsort()[:-4:-1]:
        print(f"({query_results_2[ind,i]:.6f})  {original_sentences[ind]}")
        # print(f"With weight {query_results[ind,i]:.6f}")
    print("------\n")

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight  | Sentence
(0.550276)  рим стали часто называть вечным лат
(0.079148)  также рим называют городом на семи холмах
(0.055470)  рим  один из старейших городов мира древняя столица римской империи
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight  | Sentence
(0.309441)  биологическое оружие
(0.256880)  его обвинили в заговоре с целью убийства и в нарушении нейтралитета канады но освободили под залог в 8000 долларов
(0.144803)  в 1879 году он был избран губернатором кентукки от демократической партии
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight  | Sentence
(0.553782)  флаг барбадоса  официальный символ государства барбадос
(0.105098)  посейдон со своей женой богиней амфитритой и сыном тритоном обитают в роскошном дворце на дне моря в окружении нереид гиппокампов и других обитателей моря мчится по